In [10]:
%load_ext autoreload
%autoreload 2

from mt.scan import *
from mt.ct_utils import adjust_contrast
cle.select_device("RTX")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


(OpenCL) NVIDIA GeForce RTX 4070 SUPER (OpenCL 3.0 CUDA)
	Vendor:                      NVIDIA Corporation
	Driver Version:              560.70
	Device Type:                 GPU
	Compute Units:               56
	Global Memory Size:          12281 MB
	Maximum Object Size:         3070 MB
	Max Clock Frequency:         2505 MHz
	Image Support:               Yes

In [12]:
path1 = "../../04_uCT/AD51/"
path2 = "../../04_uCT/AD67/"
scan = Scan(path2,
            low_memory_mode=0,
            discard_ends=True)
scan.load(logging=True,
          refresh=True)

Loading images from:  ../../04_uCT/AD67/Slices/
Loaded stack with shape (1781, 126, 2231) and a size of 1.00 GB in 7.14 s.
No Segmentation folder found at: ../../04_uCT/AD67/Segmentation


In [14]:
scan.try_segmentation_settings(subset_size=30)

In [5]:
settings = SegmentationSettings(otsu_sigma=1.5,
                                particle_mask_sigma=0.1,
                                particle_n_erosions=2,
                                particle_enlarge_radius=1,
                                smooth_labels_radius=2)
scan.set_segmentation_settings(settings)
print(scan.segmentation_settings)

Segmentation settings:
 Otsu thresholding: Gaussian smoothing sigma: 1.50
 Particle mask: Gaussian smoothing sigma: 0.10
 Particle mask: Erosion iterations: 2
 Particle mask: Dilation radius: 1
 Mask postprocessing: Smoothing radius: 2


In [6]:
scan.segment(logging=True)

Segmenting scan in full memory mode.


In [8]:
scan.save()

In [9]:
scan.save_segmentation()

Saving masks to:  ../../04_uCT/AD68/Segmentation


1621it [00:10, 158.92it/s]


In [7]:
scan.show()

In [9]:
viewer = napari.Viewer()
viewer.add_image(scan.stack)
viewer.add_labels(scan.mask)

<Labels layer 'Labels' at 0x1e261d70dd0>